# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [32]:
!pip install nltk

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [33]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [84]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import joblib
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

import matplotlib.pyplot as plt

In [35]:
# load data from database
engine = create_engine('sqlite:///./data/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponseTable", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [38]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [39]:
def tokenize(text):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # lemmatize, normalize case, and remove leading/trailing white space
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]

    return clean_tokens

In [40]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [41]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

joblib.dump(pipeline, './models/classifier_without_cv.pkl')

['./models/classifier_without_cv.pkl']

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [43]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x0000016233C60160>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [44]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=Y.columns.values))

precision    recall  f1-score   support

               related       0.83      0.94      0.88      5006
               request       0.79      0.55      0.65      1142
                 offer       0.00      0.00      0.00        31
           aid_related       0.76      0.60      0.67      2718
          medical_help       0.61      0.26      0.36       501
      medical_products       0.62      0.33      0.43       315
     search_and_rescue       0.63      0.22      0.33       174
              security       0.29      0.07      0.11        99
              military       0.54      0.37      0.44       190
                 water       0.74      0.63      0.68       434
                  food       0.78      0.66      0.72       767
               shelter       0.80      0.58      0.67       593
              clothing       0.68      0.49      0.57       101
                 money       0.49      0.27      0.35       143
        missing_people       0.59      0.13      0.22        75

In [45]:
y_true = pipeline.predict(X_train)
print(classification_report(y_train, y_true, target_names=Y.columns.values))

precision    recall  f1-score   support

               related       0.83      0.94      0.88     15087
               request       0.80      0.55      0.65      3332
                 offer       0.44      0.08      0.14        87
           aid_related       0.77      0.61      0.68      8142
          medical_help       0.65      0.29      0.40      1583
      medical_products       0.67      0.35      0.46       998
     search_and_rescue       0.64      0.22      0.32       550
              security       0.37      0.06      0.10       372
              military       0.67      0.36      0.47       670
                 water       0.79      0.65      0.71      1238
                  food       0.82      0.70      0.76      2156
               shelter       0.79      0.55      0.64      1721
              clothing       0.79      0.51      0.62       304
                 money       0.68      0.33      0.44       461
        missing_people       0.68      0.18      0.29       223

### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__learning_rate': [0.01, 0.1],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed: 17.7min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 21.1min finished


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x000001DCA516D3A0>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'clf__e

In [26]:
best_model = cv.best_estimator_

In [27]:
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=Y.columns.values))

precision    recall  f1-score   support

               related       0.78      1.00      0.87      5051
               request       0.84      0.40      0.55      1129
                 offer       0.00      0.00      0.00        24
           aid_related       0.77      0.44      0.56      2717
          medical_help       0.66      0.09      0.16       504
      medical_products       0.71      0.12      0.21       336
     search_and_rescue       0.67      0.07      0.13       165
              security       0.00      0.00      0.00       118
              military       0.65      0.05      0.10       208
                 water       0.76      0.54      0.64       437
                  food       0.77      0.67      0.72       707
               shelter       0.86      0.41      0.55       604
              clothing       0.70      0.30      0.42        99
                 money       0.82      0.12      0.21       152
        missing_people       0.71      0.06      0.12        79

In [28]:
y_true = best_model.predict(X_train)
print(classification_report(y_train, y_true, target_names=Y.columns.values))

precision    recall  f1-score   support

               related       0.77      1.00      0.87     15043
               request       0.84      0.39      0.53      3345
                 offer       0.00      0.00      0.00        94
           aid_related       0.79      0.45      0.57      8143
          medical_help       0.71      0.08      0.14      1580
      medical_products       0.80      0.13      0.23       977
     search_and_rescue       0.88      0.08      0.15       559
              security       0.00      0.00      0.00       353
              military       0.76      0.06      0.12       652
                 water       0.82      0.58      0.68      1235
                  food       0.78      0.71      0.74      2216
               shelter       0.85      0.40      0.55      1710
              clothing       0.81      0.32      0.46       306
                 money       0.83      0.08      0.14       452
        missing_people       0.88      0.10      0.17       219

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [34]:
import joblib
joblib.dump(best_model, 'classifier.pkl')

['classifier.pkl']

In [46]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000),
    'features__text_pipeline__tfidf__use_idf': (True, False),
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)

In [47]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 66.1min finished


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000016233C60160>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'featur

In [48]:
best_model = cv.best_estimator_

In [60]:
cv.best_params_

{'features__text_pipeline__tfidf__use_idf': True,
 'features__text_pipeline__vect__max_df': 0.75,
 'features__text_pipeline__vect__max_features': 5000,
 'features__text_pipeline__vect__ngram_range': (1, 1)}

In [49]:
joblib.dump(best_model, 'classifier2.pkl')

['classifier2.pkl']

In [50]:
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=Y.columns.values))

precision    recall  f1-score   support

               related       0.82      0.94      0.88      5006
               request       0.77      0.54      0.63      1142
                 offer       0.00      0.00      0.00        31
           aid_related       0.77      0.59      0.67      2718
          medical_help       0.63      0.28      0.39       501
      medical_products       0.62      0.32      0.42       315
     search_and_rescue       0.66      0.24      0.35       174
              security       0.38      0.08      0.13        99
              military       0.56      0.36      0.44       190
                 water       0.73      0.61      0.67       434
                  food       0.80      0.63      0.71       767
               shelter       0.80      0.56      0.66       593
              clothing       0.68      0.43      0.52       101
                 money       0.52      0.27      0.35       143
        missing_people       0.54      0.19      0.28        75

In [51]:
msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = best_model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'shelter' 'buildings' 'weather_related'
 'fire']


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [64]:
parameters = { 
 	'features__text_pipeline__vect__max_features': [5000, 3000, 1000],
 	'features__text_pipeline__vect__ngram_range': ((1,1),(1,2)),
 	'features__text_pipeline__tfidf__use_idf': (True, False)
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)

In [65]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 19.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 36.4min finished


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000016233C60160>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'featur

In [66]:
cv.best_params_

{'features__text_pipeline__tfidf__use_idf': True,
 'features__text_pipeline__vect__max_features': 1000,
 'features__text_pipeline__vect__ngram_range': (1, 1)}

In [67]:
best_model = cv.best_estimator_

In [69]:
joblib.dump(best_model, './models/classifier3.pkl')

['./models/classifier3.pkl']

In [70]:
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=Y.columns.values))

precision    recall  f1-score   support

               related       0.82      0.94      0.88      5006
               request       0.79      0.55      0.65      1142
                 offer       0.14      0.03      0.05        31
           aid_related       0.76      0.61      0.68      2718
          medical_help       0.60      0.24      0.35       501
      medical_products       0.62      0.30      0.40       315
     search_and_rescue       0.58      0.17      0.26       174
              security       0.30      0.06      0.10        99
              military       0.60      0.31      0.41       190
                 water       0.72      0.59      0.65       434
                  food       0.79      0.66      0.72       767
               shelter       0.78      0.55      0.65       593
              clothing       0.72      0.44      0.54       101
                 money       0.52      0.25      0.34       143
        missing_people       0.68      0.17      0.28        75

In [83]:
msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = best_model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'shelter' 'buildings' 'weather_related'
 'direct_report']


In [73]:
rf_pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [74]:
rf_parameters = { 
 	'features__text_pipeline__vect__max_features': [5000, 3000, 1000],
 	'features__text_pipeline__vect__ngram_range': ((1,1),(1,2)),
 	'features__text_pipeline__tfidf__use_idf': (True, False)
}

rf_cv = GridSearchCV(rf_pipeline, param_grid=parameters, verbose=2, n_jobs=-1)

In [75]:
rf_cv.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 42.7min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 78.3min finished


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000016233C60160>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=-1,
             param_grid={'fe

In [77]:
rf_cv.best_params_

{'features__text_pipeline__tfidf__use_idf': False,
 'features__text_pipeline__vect__max_features': 5000,
 'features__text_pipeline__vect__ngram_range': (1, 2)}

In [78]:
best_rf_model = rf_cv.best_estimator_

In [79]:
joblib.dump(best_rf_model, './models/rf_classifier1.pkl')

['./models/rf_classifier1.pkl']

In [81]:
y_pred = rf_cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=Y.columns.values))

precision    recall  f1-score   support

               related       0.82      0.96      0.88      5006
               request       0.85      0.51      0.64      1142
                 offer       0.00      0.00      0.00        31
           aid_related       0.76      0.65      0.70      2718
          medical_help       0.60      0.11      0.19       501
      medical_products       0.83      0.20      0.33       315
     search_and_rescue       0.73      0.09      0.16       174
              security       0.00      0.00      0.00        99
              military       0.54      0.07      0.12       190
                 water       0.84      0.49      0.62       434
                  food       0.80      0.71      0.75       767
               shelter       0.80      0.53      0.64       593
              clothing       0.66      0.23      0.34       101
                 money       1.00      0.01      0.03       143
        missing_people       0.00      0.00      0.00        75

In [82]:
msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = best_rf_model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'aid_related' 'buildings']


In [85]:
nb_pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(MultinomialNB()))
])

In [87]:
nb_parameters = { 
 	'features__text_pipeline__vect__max_features': [5000, 3000, 1000],
 	'features__text_pipeline__vect__ngram_range': ((1,1),(1,2)),
 	'features__text_pipeline__tfidf__use_idf': (True, False)
}

nb_cv = GridSearchCV(nb_pipeline, param_grid=parameters, verbose=2, n_jobs=-1)

In [88]:
nb_cv.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 16.6min finished


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000016233C60160>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())])),
                                                                       ('starting_verb',
                                                                        StartingVerbExtractor())])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=MultinomialNB()))]),
             n_jobs=-1,
             param_grid={'features__t

In [89]:
nb_cv.best_params_

{'features__text_pipeline__tfidf__use_idf': True,
 'features__text_pipeline__vect__max_features': 5000,
 'features__text_pipeline__vect__ngram_range': (1, 2)}

In [90]:
best_nb_model = nb_cv.best_estimator_

In [91]:
joblib.dump(best_nb_model, './models/nb_classifier1.pkl')

['./models/nb_classifier1.pkl']

In [92]:
y_pred = nb_cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=Y.columns.values))

precision    recall  f1-score   support

               related       0.84      0.93      0.88      5006
               request       0.71      0.68      0.70      1142
                 offer       0.00      0.00      0.00        31
           aid_related       0.73      0.65      0.69      2718
          medical_help       0.57      0.11      0.18       501
      medical_products       0.68      0.20      0.31       315
     search_and_rescue       1.00      0.01      0.02       174
              security       0.00      0.00      0.00        99
              military       0.53      0.10      0.17       190
                 water       0.76      0.26      0.39       434
                  food       0.74      0.46      0.56       767
               shelter       0.83      0.29      0.43       593
              clothing       0.41      0.09      0.15       101
                 money       1.00      0.02      0.04       143
        missing_people       0.00      0.00      0.00        75